<a href="https://colab.research.google.com/github/peremartra/fairness-pruning/blob/main/notebooks/02_Evaluate_BBQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fairness Pruning Research - Base Model Evaluation
## 02 - Comprehensive Benchmark Suite for Unpruned Models

### Establishing Performance Bias using BBVQ, EsBBQ & CatBBQ for Bias Mitigation Research
by [Pere Martra](https://github.com/peremartra)

[![GitHub](https://img.shields.io/badge/⭐_Star-OptiPFair-orange?logo=github&logoColor=white)](https://github.com/peremartra/optipfair)
[![PyPI](https://img.shields.io/pypi/v/optipfair?logo=python&logoColor=white&label=v)](https://pypi.org/project/optipfair/)

**Repository:** [github.com/peremartra/fairness-pruning](https://github.com/peremartra/fairness-pruning)

---

**Colab Environment:** GPU L4 or A100

**Models to Evaluate:**
* Llama-3.2-1B (base)
* Llama-3.2-3B (base)
* Salamandra-2B (base)
---

**Note:** This notebook evaluates ONLY base models (no pruning applied). For bias mitigation experiments with pruned models, see subsequent notebooks.

---

# 1. Setup & Installation

In [1]:
# Install required libraries
!pip install -q optipfair
!pip install -q lm-eval
!pip install -q langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Mount Google Drive for checkpoint persistence
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/fairness-pruning/main/utils.py

# Verify download
import os
if os.path.exists('utils.py'):
    print("✅ utils.py downloaded successfully")
else:
    print("❌ Failed to download utils.py")

✅ utils.py downloaded successfully


In [4]:
!wget -q https://raw.githubusercontent.com/peremartra/fairness-pruning/main/veritas_qa_ca.yaml
!wget -q https://raw.githubusercontent.com/peremartra/fairness-pruning/main/veritas_qa_es.yaml
!wget -q https://raw.githubusercontent.com/peremartra/fairness-pruning/main/veritas_lm.py

# 1. Helper Functions

Utility functions for automatic checkpoint path generation and model size detection.

# 2. Configuration & Evaluation Plan (BBQ-only)

Configure paths, select the BBQ task, and list the models we will evaluate.

In [12]:
import torch
import json
import pandas as pd
from datetime import datetime
from pathlib import Path
import logging
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display
from utils import (
    EXPERIMENT_CONFIG,
    run_robust_evaluation,
    load_or_create_model,
    clear_gpu_cache,
    format_results_table,
    get_model_stats,
 )

# Paths (Drive recommended in Colab)
CHECKPOINT_BASE_DIR = "/content/drive/MyDrive/fair_pruning/checkpoints_bbq"
RESULTS_DIR = "/content/drive/MyDrive/fair_pruning/results"
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

# BBQ task list (0-shot)
BBQ_TASKS = [{"name": "bbq", "num_fewshot": 0}]

# De-duplicate models from EXPERIMENT_CONFIG
unique_models = list(dict.fromkeys([cfg["base_model"] for cfg in EXPERIMENT_CONFIG]))

logging.getLogger("lm_eval").setLevel(logging.INFO)

print(f"{'='*70}")
print("📊 EVALUATION PLAN: BBQ-only")
print(f"{'='*70}")
print(f"Checkpoints: {CHECKPOINT_BASE_DIR}")
print(f"Results: {RESULTS_DIR}")
print(f"Tasks: {[task['name'] for task in BBQ_TASKS]}")
print("Models:")
for m in unique_models:
    print(f" - {m}")
print(f"{'='*70}\n")

📊 EVALUATION PLAN: BBQ-only
Checkpoints: /content/drive/MyDrive/fair_pruning/checkpoints_bbq
Results: /content/drive/MyDrive/fair_pruning/results
Tasks: ['bbq']
Models:
 - BSC-LT/salamandra-2b
 - meta-llama/Llama-3.2-1B
 - meta-llama/Llama-3.2-3B



In [7]:
import re
import os

def get_model_size(model_name: str) -> str:
    """Extract model size identifier from HuggingFace model name.

    Examples:
        "meta-llama/Llama-3.2-1B" → "1b"
        "meta-llama/Llama-3.2-3B-Instruct" → "3b_instruct"
        "BSC-LT/salamandra-2b" → "2b"
    """
    match = re.search(r'(\d+\.?\d*)[Bb]', model_name)
    if not match:
        return "unknown"

    size = match.group(1).replace('.', '_') + "b"
    if "instruct" in model_name.lower():
        size += "_instruct"

    return size.lower()

def get_checkpoint_path(model_name: str, base_dir: str) -> str:
    """Generate checkpoint path with size-based subdirectory.

    Args:
        model_name: Full HuggingFace model identifier
        base_dir: Base directory for checkpoints

    Returns:
        Full path to checkpoint file
    """
    model_size = get_model_size(model_name)
    safe_name = model_name.replace('/', '_').replace('-', '_').lower()
    checkpoint_dir = os.path.join(base_dir, model_size)
    os.makedirs(checkpoint_dir, exist_ok=True)
    return os.path.join(checkpoint_dir, f"{safe_name}.json")

# Test with EXPERIMENT_CONFIG
print("Testing helper functions with EXPERIMENT_CONFIG:")
print("-" * 70)
for cfg in EXPERIMENT_CONFIG:
    model_id = cfg['base_model']
    size = get_model_size(model_id)
    print(f"{model_id:<50} → {size}")
print("-" * 70)

Testing helper functions with EXPERIMENT_CONFIG:
----------------------------------------------------------------------
BSC-LT/salamandra-2b                               → 2b
meta-llama/Llama-3.2-1B                            → 1b
meta-llama/Llama-3.2-3B                            → 3b
----------------------------------------------------------------------


# 3. Run BBQ Evaluation

Evaluate each base model on the BBQ task with checkpoint/resume and raw result saving.

In [ ]:
print(f"\n{'='*70}")
print("🚀 STARTING BBQ EVALUATION (Hugging Face Native Loading)")
print(f"{'='*70}\n")

all_model_results = {}

for idx, model_id in enumerate(unique_models, 1):
    print(f"\n{'='*70}")
    print(f"📊 MODEL {idx}/{len(unique_models)}: {model_id}")
    print(f"{'='*70}\n")
    try:
        # Generar ruta de checkpoint
        checkpoint_path = get_checkpoint_path(model_id, CHECKPOINT_BASE_DIR)

        # --- CARGA DIRECTA CON HUGGING FACE ---
        print(f"📥 Loading directly from Hugging Face Hub...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

        tokenizer = AutoTokenizer.from_pretrained(model_id)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        print("✅ Model loaded successfully (Native HF)")

        # Mostrar estadísticas
        stats = get_model_stats(model)
        print(f"📈 Params: {stats['total_parameters']:,} | Size: {stats['size_gb']:.2f} GB")
        print(f"📁 Checkpoint: {checkpoint_path}\n")

        # --- EJECUTAR EVALUACIÓN (Llamada corregida) ---
        # Eliminados 'save_raw_results' y 'raw_results_dir' porque utils.py no los admite
        results = run_robust_evaluation(
            model=model,
            tokenizer=tokenizer,
            tasks=BBQ_TASKS,
            checkpoint_path=checkpoint_path,
            model_name=model_id
        )
        # -----------------------------------------------

        all_model_results[model_id] = results
        print(f"\n✅ Completed: {model_id}")
        print("Results Preview:")
        print(format_results_table(results))

    except Exception as e:
        print(f"\n❌ ERROR evaluating {model_id}: {e}")
        if 'model' in locals(): del model
        if 'tokenizer' in locals(): del tokenizer
        clear_gpu_cache()
        continue

    # Limpieza de memoria
    del model
    del tokenizer
    clear_gpu_cache()

print(f"\n{'='*70}")
print(f"✅ BBQ EVALUATION COMPLETE: {len(all_model_results)}/{len(unique_models)} models")
print(f"{'='*70}\n")

# 4. Consolidate BBQ Results

Load checkpoint files, flatten metrics, and export combined BBQ results.

In [15]:
import glob

def flatten_metrics(metrics, prefix=''):
    flat = {}
    for k, v in metrics.items():
        if isinstance(v, dict):
            flat.update(flatten_metrics(v, prefix=f"{prefix}{k}_"))
        else:
            flat[f"{prefix}{k}"] = v
    return flat

print(f"{'='*70}")
print(" CONSOLIDATING BBQ RESULTS")
print(f"{'='*70}\n")

checkpoint_files = glob.glob(f"{CHECKPOINT_BASE_DIR}/**/*.json", recursive=True)
print(f"Total checkpoint JSONs: {len(checkpoint_files)}")

consolidated_data = []

for json_path in sorted(checkpoint_files):
    if "lm_evals" in json_path:
        continue  # skip raw per-task dumps
    try:
        with open(json_path, "r") as f:
            data = json.load(f)

        metadata = data.get("metadata", {})
        model_name = metadata.get("model_name", "Unknown")

        if "1b" in model_name.lower(): model_size = "1b"
        elif "2b" in model_name.lower(): model_size = "2b"
        elif "3b" in model_name.lower(): model_size = "3b"
        else: model_size = "unknown"

        results = data.get("results", {})
        if not results:
            continue

        for task_name, metrics in results.items():
            row = {
                "model": model_name,
                "model_size": model_size,
                "task": task_name,
            }
            row.update(flatten_metrics(metrics))
            consolidated_data.append(row)

    except Exception as e:
        print(f"   -> Error reading {json_path}: {e}")

if consolidated_data:
    df = pd.DataFrame(consolidated_data)
    df = df.sort_values(by=["model", "task"]).reset_index(drop=True)
    display(df.head())

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = f"{RESULTS_DIR}/base_models_bbq_results_{timestamp}.csv"
    latest_csv = f"{RESULTS_DIR}/base_models_bbq_results_latest.csv"
    json_path = f"{RESULTS_DIR}/base_models_bbq_results_{timestamp}.json"

    df.to_csv(csv_path, index=False)
    df.to_csv(latest_csv, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print("\n💾 BBQ results saved:")
    print(f"   {csv_path}")
    print(f"   {latest_csv}")
    print(f"   {json_path}")
else:
    print("⚠️ No BBQ results found to consolidate.")

 CONSOLIDATING BBQ RESULTS

Total checkpoint JSONs: 6


,model,model_size,task,accuracy,acc_norm
0,BSC-LT/salamandra-2b,2b,bbq,0.2905,N/A
1,meta-llama/Llama-3.2-1B,1b,bbq,0.3115,N/A
2,meta-llama/Llama-3.2-3B,3b,bbq,0.4052,N/A



💾 BBQ results saved:
   /content/drive/MyDrive/fair_pruning/results/base_models_bbq_results_20251207_191101.csv
   /content/drive/MyDrive/fair_pruning/results/base_models_bbq_results_latest.csv
   /content/drive/MyDrive/fair_pruning/results/base_models_bbq_results_20251207_191101.json


# 5. Summary Analysis

Quick per-model accuracy summary for BBQ and file inventory.

In [16]:
# Summary stats
summary_df = None
if 'df' in locals() and not df.empty:
    summaries = []
    for model_name, model_df in df.groupby('model'):
        acc_series = pd.to_numeric(model_df.get('accuracy'), errors='coerce').dropna()
        summaries.append({
            "model": model_name,
            "model_size": model_df.get('model_size', pd.Series(['unknown'])).iloc[0],
            "avg_accuracy": acc_series.mean() if len(acc_series) else None,
            "tasks_completed": len(model_df)
        })

    summary_df = pd.DataFrame(summaries)
    summary_df = summary_df.sort_values("model").reset_index(drop=True)
    print(f"{'='*70}")
    print("📈 BBQ SUMMARY")
    print(f"{'='*70}")
    print(summary_df.to_string(index=False, float_format="%.4f"))

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    summary_csv = f"{RESULTS_DIR}/base_models_bbq_summary_{timestamp}.csv"
    summary_df.to_csv(summary_csv, index=False)
    print(f"\n💾 Summary saved: {summary_csv}\n")
else:
    print("No consolidated data available for summary.")

print(f"{'='*70}")
print("📁 GENERATED FILES")
print(f"{'='*70}")

if 'csv_path' in locals() and os.path.exists(csv_path):
    print(f"  ✅ {csv_path}")
if 'latest_csv' in locals() and os.path.exists(latest_csv):
    print(f"  ✅ {latest_csv}")
if 'json_path' in locals() and os.path.exists(json_path):
    print(f"  ✅ {json_path}")
if 'summary_csv' in locals() and os.path.exists(summary_csv):
    print(f"  ✅ {summary_csv}")

print("\nCheckpoints (first 10):")
if 'checkpoint_files' in locals():
    for f in sorted(checkpoint_files)[:10]:
        print(f"  ✅ {f}")
    if len(checkpoint_files) > 10:
        print(f"  ... and {len(checkpoint_files) - 10} more")

print(f"{'='*70}")

📈 BBQ SUMMARY
                  model model_size  avg_accuracy  tasks_completed
   BSC-LT/salamandra-2b         2b        0.2905                1
meta-llama/Llama-3.2-1B         1b        0.3115                1
meta-llama/Llama-3.2-3B         3b        0.4052                1

💾 Summary saved: /content/drive/MyDrive/fair_pruning/results/base_models_bbq_summary_20251207_191101.csv

📁 GENERATED FILES
  ✅ /content/drive/MyDrive/fair_pruning/results/base_models_bbq_results_20251207_191101.csv
  ✅ /content/drive/MyDrive/fair_pruning/results/base_models_bbq_results_latest.csv
  ✅ /content/drive/MyDrive/fair_pruning/results/base_models_bbq_results_20251207_191101.json
  ✅ /content/drive/MyDrive/fair_pruning/results/base_models_bbq_summary_20251207_191101.csv

Checkpoints (first 10):
  ✅ /content/drive/MyDrive/fair_pruning/checkpoints_bbq/1b/meta_llama_llama_3.2_1b.json
  ✅ /content/drive/MyDrive/fair_pruning/checkpoints_bbq/2b/bsc_lt_salamandra_2b.json
  ✅ /content/drive/MyDrive/fair_pruning